# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [ ]:
# I was having trouble loading this dataset without using encoding latin1 and using low_memory=false
df = pd.read_csv('AviationData.csv', encoding='latin1', low_memory=False)

print("First 5 rows:")
print(df.head())

print("\nDataset shape:", df.shape)

print("\nColumn names:")
print(df.columns.tolist())

print("\nData types and missing values:")
print(df.info())

print("\nMissing values per column:")
print(df.isnull().sum())

print("\nSummary statistics:")
print(df.describe(include='all'))

First 5 rows:
         Event.Id Investigation.Type Accident.Number  Event.Date  \
0  20001218X45444           Accident      SEA87LA080  1948-10-24   
1  20001218X45447           Accident      LAX94LA336  1962-07-19   
2  20061025X01555           Accident      NYC07LA005  1974-08-30   
3  20001218X45448           Accident      LAX96LA321  1977-06-19   
4  20041105X01764           Accident      CHI79FA064  1979-08-02   

          Location        Country   Latitude   Longitude Airport.Code  \
0  MOOSE CREEK, ID  United States        NaN         NaN          NaN   
1   BRIDGEPORT, CA  United States        NaN         NaN          NaN   
2    Saltville, VA  United States  36.922223  -81.878056          NaN   
3       EUREKA, CA  United States        NaN         NaN          NaN   
4       Canton, OH  United States        NaN         NaN          NaN   

  Airport.Name  ... Purpose.of.flight Air.carrier Total.Fatal.Injuries  \
0          NaN  ...          Personal         NaN               

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [14]:
df = pd.read_csv("AviationData.csv", encoding='latin1', low_memory=False)

df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')

df = df[df['Event.Date'].dt.year >= 1983]

df = df[
    df['Make'].notna() &
    df['Model'].notna() &
    df['Injury.Severity'].notna() &
    df['Aircraft.damage'].notna()
]

df = df[df['Amateur.Built'].str.upper() != 'YES']

df = df[
    df['Total.Fatal.Injuries'].notna() &
    df['Total.Serious.Injuries'].notna()
]

df['Total.Fatal.Injuries'] = pd.to_numeric(df['Total.Fatal.Injuries'], errors='coerce')
df['Total.Serious.Injuries'] = pd.to_numeric(df['Total.Serious.Injuries'], errors='coerce')

df = df.dropna(subset=['Total.Fatal.Injuries', 'Total.Serious.Injuries'])

print("Cleaned dataset shape:", df.shape)
print("Remaining columns:", df.columns.tolist())

Cleaned dataset shape: (61497, 31)
Remaining columns: ['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date', 'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 'Injury.Severity', 'Aircraft.damage', 'Aircraft.Category', 'Registration.Number', 'Make', 'Model', 'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description', 'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status', 'Publication.Date']


### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [ ]:
# Start the injury columns with 0 — This assumes if 0, no one injured in that category
df['Total.Fatal.Injuries'] = df['Total.Fatal.Injuries'].fillna(0)
df['Total.Serious.Injuries'] = df['Total.Serious.Injuries'].fillna(0)
df['Total.Minor.Injuries'] = df['Total.Minor.Injuries'].fillna(0)
df['Total.Uninjured'] = df['Total.Uninjured'].fillna(0)

# This will estimate the total number people onboard
df['Estimated.Total.Onboard'] = (
    df['Total.Fatal.Injuries'] +
    df['Total.Serious.Injuries'] +
    df['Total.Minor.Injuries'] +
    df['Total.Uninjured']
)

# This will Avoid divide-by-zero error by filtering out accidents where Estimated.Total.Onboard is 0
df = df[df['Estimated.Total.Onboard'] > 0]

# Injury rate forumula = (Fatal + Serious) / Total
df['Injury.Rate'] = (
    (df['Total.Fatal.Injuries'] + df['Total.Serious.Injuries']) /
    df['Estimated.Total.Onboard']
)

# Destruction indicator
df['Total.Destruction'] = df['Aircraft.damage'].str.strip().str.lower() == 'destroyed'
df['Total.Destruction'] = df['Total.Destruction'].astype(int)

# Confirm results
print(df[['Make', 'Model', 'Injury.Rate', 'Total.Destruction']].head())
print("\nUpdated shape:", df.shape)

        Make       Model  Injury.Rate  Total.Destruction
3601  Cessna        182P          0.0                  0
3602  Cessna       182RG          0.0                  0
3603  Cessna        182P          0.0                  0
3604   Piper  PA-28R-200          0.0                  0
3605  Cessna         140          0.0                  0

Updated shape: (61443, 34)


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [ ]:
print("Unique values in Aircraft.damage:")
print(df['Aircraft.damage'].dropna().unique())

df['Aircraft.damage'] = df['Aircraft.damage'].str.lower().str.strip()

df['Total.Destruction'] = (df['Aircraft.damage'] == 'destroyed').astype(int)

print(df['Aircraft.damage'].value_counts(dropna=False))
print(df[['Make', 'Model', 'Aircraft.damage', 'Total.Destruction']].head())

Unique values in Aircraft.damage:
['substantial' 'destroyed' 'minor' 'unknown']
Aircraft.damage
substantial    46239
destroyed      13214
minor           1901
unknown           89
Name: count, dtype: int64
        Make       Model Aircraft.damage  Total.Destruction
3601  Cessna        182P     substantial                  0
3602  Cessna       182RG     substantial                  0
3603  Cessna        182P     substantial                  0
3604   Piper  PA-28R-200     substantial                  0
3605  Cessna         140     substantial                  0


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [24]:
print("Number of unique makes:", df['Make'].nunique())
print("\nTop 20 makes by frequency:")
print(df['Make'].value_counts().head(20))

print("\nExamples of raw makes (sample):")
print(df['Make'].sample(10).tolist())

Number of unique makes: 1957

Top 20 makes by frequency:
Make
Cessna         16730
Piper           9107
CESSNA          4744
Beech           3187
PIPER           2765
Bell            1535
BEECH           1003
Grumman          869
Mooney           797
Bellanca         679
Boeing           671
Hughes           579
Robinson         571
BELL             551
Schweizer        468
Air Tractor      409
BOEING           384
Aeronca          375
Champion         351
Maule            315
Name: count, dtype: int64

Examples of raw makes (sample):
['CESSNA', 'Bell', 'Cessna', 'PIPER', 'Cessna', 'Mooney', 'Cessna', 'Piper', 'Cessna', 'CESSNA']


### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [26]:

print("Number of unique makes:", df['Make'].nunique())
print("\nTop 20 makes by frequency:")
print(df['Make'].value_counts().head(20))

print("\nExamples of raw makes (sample):")
print(df['Make'].sample(10).tolist())

df['Make'] = df['Make'].str.upper()

df['Make'] = df['Make'].str.strip()

df = df[df['Make'].notna()]

make_counts = df['Make'].value_counts()
valid_makes = make_counts[make_counts >= 50].index

df = df[df['Make'].isin(valid_makes)]

print(f"\nCleaned dataset shape: {df.shape}")
print("Remaining unique makes:", df['Make'].nunique())

Number of unique makes: 74

Top 20 makes by frequency:
Make
CESSNA               21474
PIPER                11872
BEECH                 4190
BELL                  2086
BOEING                1055
MOONEY                1032
GRUMMAN                945
BELLANCA               836
ROBINSON               820
HUGHES                 711
SCHWEIZER              607
AERONCA                524
AIR TRACTOR            497
MAULE                  459
CHAMPION               441
STINSON                359
AERO COMMANDER         342
LUSCOMBE               320
MCDONNELL DOUGLAS      309
TAYLORCRAFT            309
Name: count, dtype: int64

Examples of raw makes (sample):
['PIPER', 'PIPER', 'PIPER', 'PIPER', 'CESSNA', 'CESSNA', 'BEECH', 'CESSNA', 'BEECH', 'CESSNA']

Cleaned dataset shape: (55851, 34)
Remaining unique makes: 74


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [30]:
print("Engine.Type unique values:")
print(df['Engine.Type'].value_counts(dropna=False))

df['Engine.Type'] = df['Engine.Type'].str.upper().str.strip()

print("\nWeather.Condition unique values:")
print(df['Weather.Condition'].value_counts(dropna=False))

df['Weather.Condition'] = df['Weather.Condition'].str.upper().str.strip()

print("\nNumber.of.Engines value counts:")
print(df['Number.of.Engines'].value_counts(dropna=False).sort_index())

df = df[df['Number.of.Engines'] >= 1]

print("\nPurpose.of.flight unique values:")
print(df['Purpose.of.flight'].value_counts(dropna=False))

df['Purpose.of.flight'] = df['Purpose.of.flight'].str.upper().str.strip()

print("\nBroad.phase.of.flight unique values:")
print(df['Broad.phase.of.flight'].value_counts(dropna=False))

df['Broad.phase.of.flight'] = df['Broad.phase.of.flight'].str.upper().str.strip()

print("\nFinal cleaned column values:")
print("Engine.Type:", df['Engine.Type'].unique())
print("Weather.Condition:", df['Weather.Condition'].unique())
print("Purpose.of.flight:", df['Purpose.of.flight'].unique())
print("Broad.phase.of.flight:", df['Broad.phase.of.flight'].unique())

Engine.Type unique values:
Engine.Type
RECIPROCATING      45863
TURBO SHAFT         2294
TURBO PROP          1975
NaN                 1557
TURBO FAN            865
TURBO JET            254
UNKNOWN               31
GEARED TURBOFAN        1
UNK                    1
Name: count, dtype: int64

Weather.Condition unique values:
Weather.Condition
VMC    47334
IMC     4089
NaN      841
UNK      577
Name: count, dtype: int64

Number.of.Engines value counts:
Number.of.Engines
1.0    45574
2.0     6789
3.0      238
4.0      240
Name: count, dtype: int64

Purpose.of.flight unique values:
Purpose.of.flight
PERSONAL                     29661
INSTRUCTIONAL                 7415
UNKNOWN                       3984
AERIAL APPLICATION            3487
BUSINESS                      2866
NaN                           1256
POSITIONING                   1019
OTHER WORK USE                 780
AERIAL OBSERVATION             565
FERRY                          540
PUBLIC AIRCRAFT                401
EXECUTIVE/CORP

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [32]:
# Going to drop columns where over 50% data is missing
missing_percent = df.isnull().mean() * 100
print(missing_percent.sort_values(ascending=False))

threshold = 50
cols_to_drop = missing_percent[missing_percent > threshold].index.tolist()

df.drop(columns=cols_to_drop, inplace=True)

print(f"\nDropped columns: {cols_to_drop}")
print(f"Updated shape: {df.shape}")
print(f"Remaining columns: {df.columns.tolist()}")

Schedule                   86.633485
Air.carrier                80.484851
Latitude                   72.139059
Longitude                  72.139059
FAR.Description            72.072822
Aircraft.Category          71.991446
Airport.Code               41.390208
Airport.Name               38.299805
Broad.phase.of.flight      28.241328
Publication.Date           20.559793
Report.Status               4.390530
Engine.Type                 2.946576
Purpose.of.flight           2.376942
Weather.Condition           1.591567
Country                     0.310365
Location                    0.022710
Registration.Number         0.020817
Total.Serious.Injuries      0.000000
Total.Uninjured             0.000000
Total.Minor.Injuries        0.000000
Estimated.Total.Onboard     0.000000
Injury.Rate                 0.000000
Event.Id                    0.000000
Number.of.Engines           0.000000
Total.Fatal.Injuries        0.000000
Investigation.Type          0.000000
Amateur.Built               0.000000
M

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [33]:
df.to_csv("Cleaned_AviationData.csv", index=False)

print("Cleaned data saved to 'Cleaned_AviationData.csv'")

Cleaned data saved to 'Cleaned_AviationData.csv'
